In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


-----

#### tsfel + 4 
change window sliding seconds (3, 5, 10)

In [ ]:
pip install tsfel

In [ ]:
# TSFEL + 4 
import os, re, glob, warnings
import numpy as np
import pandas as pd
from collections import defaultdict

from scipy.signal import welch
from scipy.stats import entropy

from sklearn.model_selection import train_test_split, GroupKFold, GridSearchCV, LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, f1_score, balanced_accuracy_score,
                             precision_score, recall_score, classification_report,
                             confusion_matrix, roc_auc_score)

import matplotlib.pyplot as plt
import seaborn as sns
import tsfel

warnings.filterwarnings("ignore")

# settings
FS = 40
WINDOW_SEC = 10
OVERLAP = 0.5
BASE_IN = "/content/drive/My Drive/final_project/smooth_data4"
BASE_OUT = f"/content/drive/My Drive/final_project/segment_{WINDOW_SEC}s_tsfel4"
os.makedirs(BASE_OUT, exist_ok=True)

def parse_fileinfo(path):
    name = os.path.basename(path).replace(".csv","")
    sensor = "wrist" if re.search(r"wrist", name, re.I) else ("ankle" if re.search(r"ankle", name, re.I) else None)
    m = re.search(r"T(\d+)_((M10[_-]?WT\d+)|M\d+)", name, re.I)
    if not m or sensor is None:
        return None
    subject = int(m.group(1))
    activity = m.group(2).replace("_","-").upper().replace("M10-WT","M10_WT")
    return subject, activity, sensor

def spectral_centroid(f, pxx):
    pxx = np.clip(pxx, 1e-12, None)
    return float(np.sum(f * pxx) / np.sum(pxx))

def spectral_edge_freq(f, pxx, edge=0.9):
    p = np.cumsum(np.clip(pxx, 1e-12, None))
    p = p / p[-1]
    idx = np.searchsorted(p, edge)
    return float(f[min(idx, len(f)-1)])

def tsfel_feats_for_series(series, cfg):
    df_tmp = tsfel.time_series_features_extractor(cfg, series.reset_index(drop=True), fs=FS, verbose=0)
    return df_tmp.iloc[0].to_dict()

def jerk_features_from_acc_mag(win_df, prefix=""):
    need = ["Acc_X","Acc_Y","Acc_Z"]
    if not all([c in win_df.columns for c in need]):
        return {}
    ax, ay, az = [win_df[c].values.astype(float) for c in need]
    amag = np.sqrt(ax*ax + ay*ay + az*az)
    jerk = np.diff(amag) * FS
    if len(jerk) == 0:
        return {}
    # jerk 
    j_mean = float(np.mean(jerk))
    j_std  = float(np.std(jerk, ddof=1)) if len(jerk)>1 else 0.0
    # 频域特征
    f, pxx = welch(amag, fs=FS, nperseg=min(len(amag), 256))
    sc = spectral_centroid(f, pxx)
    sef90 = spectral_edge_freq(f, pxx, edge=0.9)
    return {
        f"{prefix}jerk_mean_mag": j_mean,
        f"{prefix}jerk_std_mag": j_std,
        f"{prefix}spectral_centroid_acc_mag": sc,
        f"{prefix}spectral_edge_freq90_acc_mag": sef90
    }

def segment_and_extract_tsfel(grouped, window_size, step):
    cfg = tsfel.get_features_by_domain() 
    rows = []
    one_d_cols = [
        "Acc_X","Acc_Y","Acc_Z",
        "Gyr_X","Gyr_Y","Gyr_Z",
        "FreeAcc_E","FreeAcc_N","FreeAcc_U",
        "Roll","Pitch","Yaw"
    ]
    for (subj, act), sensors in grouped.items():
        if "wrist" not in sensors or "ankle" not in sensors:
            continue
        df_w = sensors["wrist"]; df_a = sensors["ankle"]
        L = min(len(df_w), len(df_a))
        if L < window_size:
            continue
        for widx, start in enumerate(range(0, L - window_size + 1, step), 1):
            win_w = df_w.iloc[start:start+window_size]
            win_a = df_a.iloc[start:start+window_size]
            feats = {}
            # Ankle TSFEL
            for col in one_d_cols:
                if col in win_a.columns:
                    d = tsfel_feats_for_series(win_a[col], cfg)
                    feats.update({f"ankle_{col}__{k}": float(v) for k,v in d.items()})
            feats.update(jerk_features_from_acc_mag(win_a, prefix="ankle_"))
            # Wrist TSFEL
            for col in one_d_cols:
                if col in win_w.columns:
                    d = tsfel_feats_for_series(win_w[col], cfg)
                    feats.update({f"wrist_{col}__{k}": float(v) for k,v in d.items()})
            feats.update(jerk_features_from_acc_mag(win_w, prefix="wrist_"))

            feats["window"] = widx
            feats["Subject"] = subj
            feats["Activity"] = act
            rows.append(feats)
    return pd.DataFrame(rows)

def ensure_dirs():
    os.makedirs(f"{BASE_OUT}/features", exist_ok=True)
    os.makedirs(f"{BASE_OUT}/gender", exist_ok=True)
    os.makedirs(f"{BASE_OUT}/gender_merged", exist_ok=True)
    os.makedirs(f"{BASE_OUT}/figures", exist_ok=True)

# read file
ensure_dirs()
files = sorted(glob.glob(os.path.join(BASE_IN, "*.csv")))
grouped = defaultdict(dict)
for fp in files:
    info = parse_fileinfo(fp)
    if info is None:
        continue
    s, a, sensor = info
    df = pd.read_csv(fp)
    grouped[(s,a)][sensor] = df


In [ ]:

# window sliding & TSFEL+4 features extraction
window_size = int(WINDOW_SEC * FS)
step = max(1, int(window_size * (1-OVERLAP)))  # 50% overlap
features_df = segment_and_extract_tsfel(grouped, window_size, step)
feat_path = f"{BASE_OUT}/features/all_features_merged.csv"
features_df.to_csv(feat_path, index=False)
print("Saved:", feat_path, "shape:", features_df.shape)

# merge demographicss 
demo_xlsx = "/content/drive/My Drive/final_project/participant_demographics_table.xlsx"
demo = pd.read_excel(demo_xlsx)
demo.columns = demo.columns.str.strip().str.replace(" ", "_").str.lower()
if "trial_id" in demo.columns:
    demo["subject"] = demo["trial_id"].str.extract(r"T(\d+)", expand=False).astype(int)
elif "subject" in demo.columns and demo["subject"].dtype == object:
    demo["subject"] = demo["subject"].str.extract(r"T?(\d+)", expand=False).astype(int)

merged = features_df.merge(demo, left_on="Subject", right_on="subject", how="left")
merged_path = f"{BASE_OUT}/features/features_with_demo.csv"
merged.to_csv(merged_path, index=False)

act_counts = merged.groupby("Subject")["Activity"].nunique().reset_index()
full_subjects = act_counts[act_counts["Activity"] >= 12]["Subject"].tolist()

subj_df = merged.groupby("Subject").agg(
    sex=("sex","first"),
    stage=("hoehn_and_yahr_(stage)","first")
).reset_index()
subj_df["sex"] = subj_df["sex"].fillna("Unknown").astype(str)
subj_df["stage"] = subj_df["stage"].fillna("Unknown").astype(str)

full_df = subj_df[subj_df["Subject"].isin(full_subjects)]
n_total = subj_df["Subject"].nunique()
n_test  = max(1, round(n_total * 0.2))
if len(full_df) >= n_test:
    test_subj_df, _ = train_test_split(full_df, test_size=len(full_df)-n_test,
                                       stratify=full_df["sex"], random_state=42)
else:
    test_subj_df, _ = train_test_split(subj_df, test_size=n_total-n_test,
                                       stratify=subj_df["sex"], random_state=42)

remain = subj_df[~subj_df["Subject"].isin(test_subj_df["Subject"])]
train_subj_df, val_subj_df = train_test_split(
    remain, test_size=0.25, stratify=remain["sex"], random_state=42
)

df = merged.copy()
df["set"] = "train"
df.loc[df["Subject"].isin(val_subj_df["Subject"]), "set"] = "val"
df.loc[df["Subject"].isin(test_subj_df["Subject"]), "set"] = "test"
df.to_csv(f"{BASE_OUT}/gender/features_with_demo_split.csv", index=False)

m10_map = {f"M10_WT{i}":"M10" for i in range(6)}
for split in ["train","val","test"]:
    part = df[df["set"]==split].copy()
    part["Activity"] = part["Activity"].replace(m10_map)
    part.to_csv(f"{BASE_OUT}/gender_merged/{split}.csv", index=False)


In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

# train/ val & LOSO
train_df = pd.read_csv(f"{BASE_OUT}/gender_merged/train.csv")
val_df   = pd.read_csv(f"{BASE_OUT}/gender_merged/val.csv")

exclude_cols = [
    "Activity","Subject","window","set","trial_id","subject",
    "sex","most_affected_wrist","most_affected_ankle","dominant_side",
    "age","height_(cm)","weight_(kg)","most_affected_side",
    "hoehn_and_yahr_(stage)","years_since_diagnosis","cit","updrs"
]
feature_cols = [c for c in train_df.columns if c not in exclude_cols]

X_train, y_train = train_df[feature_cols], train_df["Activity"]
X_val,   y_val   = val_df[feature_cols],   val_df["Activity"]

d = X_train.shape[1]

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("kbest", SelectKBest(score_func=f_classif)),
    # ("clf", RandomForestClassifier(class_weight="balanced", random_state=42))
    ("clf", HistGradientBoostingClassifier(
        random_state=42,
        early_stopping=True,  
        validation_fraction=0.1,  
        n_iter_no_change=10
    ))

])

# RF
# param_grid = {
#     "kbest__k": [100, 150, 200],    
#     "clf__n_estimators": [200, 250],
#     "clf__max_depth": [2, 3, 5, 10],
#     "clf__min_samples_leaf": [2, 4, 8],
#     "clf__min_samples_split": [24, 8],
#     "clf__max_features": ["sqrt","log2"],     
#     "clf__bootstrap": [True], 
#     "clf__ccp_alpha": [0.1] 
# }

# HGB
param_grid = {
    "kbest__k": [70, 100, 150, 200],
    "clf__learning_rate": [0.001, 0.005, 0.01, 0.05, 0.1, 0.2],   
    "clf__max_iter": [100, 200, 300],   
    "clf__max_depth": [2, 3, 5, 7, 10],  
    "clf__min_samples_leaf": [2, 3, 10, 30], 
    "clf__l2_regularization": [0.1, 0.5, 10.0, 20.0, 60.0], 
    "clf__max_bins": [128, 255],
}


gkf = GroupKFold(n_splits=3)
gs = GridSearchCV(pipe, param_grid, cv=gkf, scoring="accuracy", n_jobs=-1, verbose=1)
gs.fit(X_train, y_train, groups=train_df["Subject"])
print("Best params:", gs.best_params_)
best = gs.best_estimator_

def all_metrics(model, X_tr, y_tr, X_va, y_va, name_prefix=""):
    ytr_pred = model.predict(X_tr)
    yva_pred = model.predict(X_va)

    yva_proba = model.predict_proba(X_va) if hasattr(model,"predict_proba") else None

    metrics = {}
    metrics[f"{name_prefix}Train Accuracy"] = accuracy_score(y_tr, ytr_pred)
    metrics[f"{name_prefix}Train Macro F1"] = f1_score(y_tr, ytr_pred, average="macro")
    metrics[f"{name_prefix}Train Balanced Accuracy"] = balanced_accuracy_score(y_tr, ytr_pred)
    metrics[f"{name_prefix}Train Macro Precision"] = precision_score(y_tr, ytr_pred, average="macro", zero_division=0)

    metrics[f"{name_prefix}Val Accuracy"] = accuracy_score(y_va, yva_pred)
    metrics[f"{name_prefix}Val Macro F1"] = f1_score(y_va, yva_pred, average="macro")
    metrics[f"{name_prefix}Val Balanced Accuracy"] = balanced_accuracy_score(y_va, yva_pred)
    metrics[f"{name_prefix}Val Macro Precision"] = precision_score(y_va, yva_pred, average="macro", zero_division=0)

    if yva_proba is not None:
        lb = LabelBinarizer().fit(y_va)
        y_true_bin = lb.transform(y_va)
        try:
            metrics[f"{name_prefix}Val Macro ROC-AUC"] = roc_auc_score(y_true_bin, yva_proba, average="macro", multi_class="ovr")
        except Exception:
            metrics[f"{name_prefix}Val Macro ROC-AUC"] = np.nan

    print(pd.Series(metrics).to_string())
    print("\nValidation Classification Report:")
    print(classification_report(y_va, yva_pred))

    labels = sorted(y_va.unique())
    cm = confusion_matrix(y_va, yva_pred, labels=labels)
    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)
    plt.title("Confusion Matrix (Validation)")
    plt.xlabel("Predicted"); plt.ylabel("True")
    plt.xticks(rotation=45, ha="right"); plt.yticks(rotation=0)
    plt.tight_layout()
    fig_path = f"{BASE_OUT}/figures/cm_val_{WINDOW_SEC}s.png"
    os.makedirs(os.path.dirname(fig_path), exist_ok=True)
    plt.savefig(fig_path, dpi=150)
    plt.show()
    print("Saved:", fig_path)
    return metrics

best.fit(X_train, y_train)
_ = all_metrics(best, X_train, y_train, X_val, y_val)

# LOSO
comb = pd.concat([train_df, val_df], ignore_index=True)
X = comb[feature_cols]; y = comb["Activity"]; groups = comb["Subject"]
logo = LeaveOneGroupOut()

fold_acc, fold_f1, fold_balacc = [], [], []
y_true_all, y_pred_all = [], []

for tr_idx, va_idx in logo.split(X, y, groups):
    Xtr, Xva = X.iloc[tr_idx], X.iloc[va_idx]
    ytr, yva = y.iloc[tr_idx], y.iloc[va_idx]

    loso_model = Pipeline([
        ("scaler", best.named_steps["scaler"]),
        ("kbest", SelectKBest(f_classif, k=best.named_steps["kbest"].k)),
        # ("clf", RandomForestClassifier(
        #     n_estimators=best.named_steps["clf"].n_estimators,
        #     max_depth=best.named_steps["clf"].max_depth,
        #     min_samples_leaf=best.named_steps["clf"].min_samples_leaf,
        #     max_features=best.named_steps["clf"].max_features,
        #     class_weight="balanced",
        #     random_state=42
        # ))
        ("clf", HistGradientBoostingClassifier(
        learning_rate=best.named_steps["clf"].learning_rate,
        max_iter=best.named_steps["clf"].max_iter,
        max_depth=best.named_steps["clf"].max_depth,
        min_samples_leaf=best.named_steps["clf"].min_samples_leaf,
        l2_regularization=best.named_steps["clf"].l2_regularization,
        max_bins=best.named_steps["clf"].max_bins,
        early_stopping=True, 
        validation_fraction=0.1,  
        n_iter_no_change=10,
        random_state=42
        ))
    ])
    loso_model.fit(Xtr, ytr)
    yp = loso_model.predict(Xva)

    fold_acc.append(accuracy_score(yva, yp))
    fold_f1.append(f1_score(yva, yp, average="macro"))
    fold_balacc.append(balanced_accuracy_score(yva, yp))
    y_true_all.extend(yva); y_pred_all.extend(yp)

print(f"LOSO Accuracy: {np.mean(fold_acc):.4f} ± {np.std(fold_acc):.4f}")
print(f"LOSO Macro F1: {np.mean(fold_f1):.4f} ± {np.std(fold_f1):.4f}")
print(f"LOSO Balanced Acc: {np.mean(fold_balacc):.4f} ± {np.std(fold_balacc):.4f}")

labels = sorted(comb["Activity"].unique())
cm = confusion_matrix(y_true_all, y_pred_all, labels=labels)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)
plt.title(f"LOSO Confusion Matrix ({WINDOW_SEC}s windows)")
plt.xlabel("Predicted"); plt.ylabel("True")
plt.xticks(rotation=45, ha="right"); plt.yticks(rotation=0)
plt.tight_layout()
loso_fig = f"{BASE_OUT}/figures/cm_loso_{WINDOW_SEC}s.png"
plt.savefig(loso_fig, dpi=150)
plt.show()
print("Saved:", loso_fig)
